<a href="https://colab.research.google.com/github/pjhool/Deep-Learning-with-Keras/blob/master/Keras_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
from keras.layers import Input
from keras.layers.core import Dense
from keras.models import Model
from keras.layers.core import Activation

inputs = Input(shape=(784,))

x = Dense(32)(inputs)
x = Activation("sigmoid")(x)
x = Dense(10)(x)
predictions = Activation("softmax")(x)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss="categorical_crossentropy", optimizer="adam")


In [0]:
from keras.layers import Input
from keras.layers.core import Dense
from keras.models import Model
from keras.layers.core import Activation

input1 = Input(shape=(784,))
input2 = Input(shape=(784,))

x = Dense(32)(input1)
x = Activation("sigmoid")(x)
x = Dense(10)(x)
output1 = Activation("softmax")(x)

x = Dense(32)(input2)
x = Activation("sigmoid")(x)
x = Dense(10)(x)
output2 = Activation("softmax")(x)

model = Model(inputs=[input1, input2], outputs=[output1, output2])

model.compile(loss="categorical_crossentropy", optimizer="adam")
© 2019 GitHub, Inc.